# Chapter 7: Datasets

[**7.1 Datasets**](#7.1-Datasets)   
[**7.2 Creating Datasets**](#7.2-Creating-Datasets)   
[**7.2.1 Scala**](#7.2.1-Scala)   
[**7.2.2 Java**](#7.2.2-Java)   
[**7.3 DataFrame and Dataset**](#7.3-DataFrame-and-Dataset)   
[**7.4 Encoder**](#7.4-Encoder)   
[**7.5 Actions**](#7.5-Actions)   
[**7.6 Transformations**](#7.6-Transformations)   
[**7.7 Joins**](#7.7-Joins)   
[**7.8 Grouping and Aggregations**](#7.8-Grouping-and-Aggregations)     
[**7.9 Write Output to File**](#7.9-Write-Output-to-File)  

#### 7.1 Datasets
We have already learned two structured API i.e. DataFrame and SparkSQL. Datasets are the foundational type for all the structured APIs. It is Java Virtual Machine (JVM) language feature that works only with Scala and Java. Datasets are language-native type classes and objects in Scala and Java, where DataFrame doesn't have those characteristic.

`Datasets` is a strongly-typed collection of domain-specific objects that can be tranformed in parallel using functional or relational operations. Datasets supports both `strongly-typed` and `untyped` API. As said earlier, Datasets only exists in Scala and Java. DataFrames exists in Python and R. Comparing with DataFrame, DataFrames are Datasets of type `Row`. Row is a generic typed JVM object that holds different types of fields. Spark will internally convert the Row objects into Spark types. For e..g Int Row will be convered to IntegerType and IntegerType() in Scala/Java and Python respectively. 

**Needs of Datasets**  
* If the functionality are not supported in DataFrame.
* If type-safety is the major issue.

If we want to convert a large business logic then instead of using DataFrame and SparkSQL we can use Datasets. Something if we are working on calculation and we want the output be precise then we can use Dataset. Datasets are used if we want to reuse multiple transformations of entire rows between single-node workloads and Spark workloads. i.e. During ETL, if we want to collect data by driver and manipulate in single-node libaries at the beginning of transformation.  


Let walk through the concept of Dataset described in Databricks [articles](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html). 

**_Self Reading_**: Read the above article and this [article](https://databricks.com/blog/2016/01/04/introducing-apache-spark-datasets.html) and identify the needs and usage of using Datasets.


**Dataset APIs characteristics**  
* Strongly-typed API
* Untype API

According to Databrick's blog: If we consider, DataFrame as an alias for a collection of generic objects `Dataset[Row]`, `Row` is a generic `untyped` JVM object. Dataset is a collection of `strongly-typed` JVM objects.

![Databrick Dataset APIs](https://databricks.com/wp-content/uploads/2016/06/Unified-Apache-Spark-2.0-API-1.png)

#### 7.2 Creating Datasets

While creating Datasets we need to define the schemas at the beginning. We can define the object for all the row in Dataset using Scala or Java.  
* `case class` object is used in Scala for defining a schema.
* `JavaBean` is used in Java for defining a schema.

**7.2.1 Scala**  
In Scala, we need to create `case class`. `case class` is a regular class which are used for modeling immutable data and pattern matching.  
[Check out for case class](https://docs.scala-lang.org/tour/case-classes.html).

**Demo on spark-shell**  

Step 1: Run spark-shell in terminal  
Step 2: Create dummy dataset

Filename: /tmp/dataset.csv  
1,Bob,A  
2,Harry,A  
3,John,A  

_Create case class in Scala for employee_

In [ ]:
// define a case class that represent employee data
case class Employee(
    emp_id: Int,
    name: String,
    grade: String  //pay grade
)

_Read the employee data and create Dataset[Row] from `case class` Employee_

In [ ]:
import org.apache.spark.sql.Encoders

val emp = spark.read.schema(Encoders.product[Employee].schema).csv("/tmp/dataset.csv").as[Employee]

// It will return:
emp: org.apache.spark.sql.Dataset[Employee] = [student_id: int, name: string ... 1 more field]

_display employee record_

In [ ]:
emp.first.emp_id // it will return first emp_id as well as its type

In [ ]:
emp.first.name // it will return first name as well as its type

_Print top 10 rows_

In [ ]:
emp.take(10).foreach(println(_))

It will return:  
Employee(1,Bob,A)  
Employee(2,Harry,A)  
Employee(3,John,A)  

**7.2.2 Java**  
In Java, we need to create our class and encode the DataFrame.

**Demo**  

https://databricks.com/spark/getting-started-with-apache-spark/datasets

In [ ]:
import org.apache.spark.sql.Encoders;
import java.io.Serializable;

public class Employee implements Serializable{
 int id ;
 String firstname;
 String lastname;
 Datetime dob;
    
 //getter and setters
 int getId(){
     return id;
 }
 
 String getFirstName{
     return firstname;
 }
 @todo setter
}

Dataset<Employee> emp = spark.read
  .csv("/tmp/employees.csv")
  .as(Encoders.bean(Employee.class));

#### 7.3 DataFrame and Dataset

There are some reason of using DataFrame and Dataset.  


---------------------
`DataFrame`
* For processing relational transformation similar to SQL like queries.
* For unification, code optimization, simplification of APIs across Spark Libraries.
---------------------
`Dataset` 
* For strict compile type safety, where multiple case classes need to be created for specific Dataset[T].
* For higher degree of type-safety at compile time.
* For typed JVM objects by taking advantage of Catalyst optimization and benefiting from Tungsten’s efficient code generation and serialization with Encoders.
---------------------
`DataFrame/Dataset`
* For rich semantics, high-level abstractions, and domain specific language operators.
* For processing demands high-level expressions, filters, maps, aggregation, averages, sum, SQL queries, columnar access and use of relational operators on semi-structured data.
---------------------

#### 7.4 Encoder
Encoder is the fundamental concept in the serialization and deserialization (SerDe) framework in Spark. It is also known as container of serde expressions in Dataset. Encoder is used to map the domain-specific type to Spark's internal type. It convert in-memory`off Java heap` data from Spark's Tungsten format to JVM Java objects. Basically, it serializes and deserializes Dataset objects from Spark's internal format to JVM objects.

For example, In `Employee` class that has two field, `id` int and `name` string, Encoder will serialize the `Employee` object into binary structure. In Structure APIs, the binary structure is known as `Row`. While using Dataset API, all the row will be converted into object. 

Spark has built-in feature for generating `Encoders` for primitive types, Scalas case classes, and Java Beans. It's encoder is faster than Java and Kyro serde. [For more info](https://databricks.com/blog/2016/01/04/introducing-apache-spark-datasets.html)  

Spark uses [Tungsten](https://databricks.com/glossary/tungsten) for memory management. Tungsten stores objects off the Java heap memory which is compact and occupy less space compared to Java storage. Lets see the simple example in [Java Vs Spark Tungsten row-based format](https://spoddutur.github.io/spark-notes/deep_dive_into_storage_formats.html). Encoder will quickly serialize and deserialize using pointer arithmetic with memory address and offset. The figure below shows the comparision with Encoder, Java, and Kyro SerDes. ![Compare Encoder with Java and Kyro](https://databricks.com/wp-content/uploads/2016/01/Serialization-Deserialization-Performance-Chart-1024x364.png?noresize) I highly suggest to read the Spark storage format in this [link](https://spoddutur.github.io/spark-notes/deep_dive_into_storage_formats.html).

#### 7.5 Actions
We can apply all the actions such as `collect, take, count` etc. in Datasets. [More info](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset)

In [ ]:
emp.show(5)

In [ ]:
+----------+--------+-----+  
|student_id|    name|grade|  
+----------+--------+-----+  
|         1|     Bob|    A|  
|         2|   Harry|    A|  
|         3|    John|    A|  
+----------+--------+-----+  

**Accessing element from `case class`**  
Specifying the attribute name will return the values as well as data type. 

In [ ]:
emp.first.name // it will return first element from the DataFrame

#### 7.6 Transformations 
All the transformations in DataFrame are supported by Datasets. We can also use complex and strongly type transformation in Datasets. We need to define the `generic function` for any new transformation. These function is not a UDF. All the function can be tested locally before executing to Spark.  [More info](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset)

_Filtering_  
We can create filter by defining generic function. The function `gradeA` will check if the grade is A. It will return Boolean.

In [ ]:
def gradeA(emp_record: Employee): Boolean = {
    return emp_record.grade == "A"
}

In [ ]:
emp.filter(emp_record => gradeA(emp_record)).first()

It will return  
Employee = Employee(1,Bob,A)

In [ ]:
emp.filter(emp_record => gradeA(emp_record)).show()

_Mapping_  
Mapping is used when we want to map one value to other value. For e.g. extracting value, comparing values etc. The example below shows extracting one value from each row. It is similar to `select` in DataFrame. 

In [ ]:
val full_name = emp.map(f => f.name)

In [ ]:
full_name.show()

In [ ]:
It will return
+--------+
|   value|
+--------+
|     Bob|
|   Harry|
|    John|
+--------+

#### 7.7 Joins
The joins used in DataFrame can also be applied to Datasets. Datasets has `joinWith` method which is similar to co-group in RDD. It will return two nested Datasets inside of one. `joinWith` creates a Dataset with two columns `_1` and `_2` that matches with the specified condition. It is used when we want to join and apply advance manipulation on result such as advance map or filter.

In [ ]:
case class EmployeeList(sequence_num: BigInt, id: Int)

val employee_list = spark.range(10).map(x => (x, scala.util.Random.nextInt(50)))
.withColumnRenamed("_1", "sequence_num")
.withColumnRenamed("_2", "id").as[EmployeeList]

In [ ]:
employee_list
// It returns
employee_list: org.apache.spark.sql.Dataset[EmployeeList] = [sequence_num: bigint, id: int]

In [ ]:
val employee_info = emp.joinWith(employee_list, emp.col("emp_id") === employee_list.col("sequence_num"))

In [ ]:
employee.show()
employee_list.show()
employee_info.show()

**Example 2**  
[Self Reading](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-joins.html#joinWith) 

In [ ]:
case class Student(id: Long, name: String, state_id: Int)

In [ ]:
case class State(s_id: Int, state_name: String, city: String)

In [ ]:
val stu = Seq(
    Student(1,"John",10),
    Student(2,"Harry",20),
    Student(3,"Bob",30),
    Student(4,"Michael",10),
    Student(5,"Travis",20)    
).toDS

In [ ]:
val state = Seq(
    State(10,"VA","Glen Allen"),
    State(20,"MA","Harvard"),
    State(30,"TX","Irvine"),
    State(10,"VA","Henrico"),
    State(10,"VA","Herndon")    
).toDS

In [ ]:
val join_stu_state = stu.joinWith(state, stu("state_id") === state("s_id"))

In [ ]:
join_stu_state.printSchema

In [ ]:
stu.show()
state.show()
join_stu_state.show()

**Explode struct type in Dataset**

#### 7.8 Grouping and Aggregations
We can apply the grouping and aggreations similar to DataFrame in Datasets but it will return DataFrame type. So, the information on type will be lost. To keep the same type we can apply several methods. For e.g. `groupByKey` method will allow to perform group by a specific key in the Dataset and it will also return type Dataset.

[More info](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset)

In [ ]:
emp.groupByKey(e => e.grade).count()

It return  
org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]

In [ ]:
state.groupByKey(e => e.s_id).count().show()

In [ ]:
emp.groupByKey(e => e.grade).count().show()

In [ ]:
emp.groupByKey(e => e.grade).count().explain()

#### 7.9 Write Output to File

We can write the output to external storage using `write()` method. It uses DataFrameWriter interface to write a Dataset to external storage systems. We can save to multiple format such as csv, json, parquet, text etc as well as jdbc. [More information about saving output](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-DataFrameWriter.html).

In [ ]:
// write to json file
emp.write.format("json").save("/tmp/employees/dataset/")

In [ ]:
// write to csv file
emp.write.format("csv").save("/tmp/employees/dataset1/")

**Further reading**  
[More information on Dataset API Operators](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-dataset-operators.html)